<h1>Знакомство с базой данных</h1>

Проект состоит из двух частей:
В первой части вы решите несколько задач в SQL-тренажёре, чтобы закрепить пройденный материал.
Вторая часть проекта — аналитическая. Проверять задачи по-прежнему будет тренажёр. Однако мы рекомендуем самостоятельно анализировать полученные результаты и формулировать выводы.

Внимательно читайте условия. Формулировки задач в проекте приближены к реальной жизни. В работе часто встретятся такие формулировки: «рассчитать LTV за последние 30 дней», «отобразить самых активных покупателей» или даже «определить, почему в компании упали продажи».

В самостоятельном проекте вы будете работать с базой данных StackOverflow — сервиса вопросов и ответов о программировании. StackOverflow похож на социальную сеть — пользователи сервиса задают вопросы, отвечают на посты, оставляют комментарии и ставят оценки другим ответам.
Вы будете работать с версией базы, где хранятся данные о постах за 2008 год, но в таблицах вы найдёте информацию и о более поздних оценках, которые эти посты получили.
Изучите ER-диаграмму базы:

<h2>Первая часть</h2>

Схема БД: <https://ibb.co/QDB9cJj>

<strong>1.
Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».</strong>

SELECT COUNT(p.id)

FROM stackoverflow.posts p JOIN stackoverflow.post_types pt ON p.post_type_id = pt.id

WHERE type = 'Question'

  AND (favorites_count >= 100
  
   OR score > 300);



<strong>2.
Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.</strong>

WITH

questions as (SELECT creation_date::DATE,

   COUNT(p.id)
   
FROM stackoverflow.posts p JOIN stackoverflow.post_types pt ON p.post_type_id = pt.id

WHERE type = 'Question'

    AND creation_date::date between '2008-11-01' and '2008-11-18'
    
GROUP BY creation_date::DATE)


SELECT ROUND(AVG(count))

FROM questions;

<strong>3.
Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.</strong>

SElECT COUNT(DISTINCT u.id)

FROM stackoverflow.users as u

JOIN stackoverflow.badges as b ON u.id = b.user_id

WHERE u.creation_date::date = b.creation_date::date;

<strong>
4.
Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?</strong>

SELECT COUNT(DISTINCT p.id)

FROM stackoverflow.posts p JOIN stackoverflow.users u ON u.id = p.user_id JOIN stackoverflow.votes v ON v.post_id = p.id

WHERE display_name = 'Joel Coehoorn'

<strong>5.
Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.</strong>

SElECT *, ROW_NUMBER() OVER (ORDER BY id DESC )

FROM stackoverflow.vote_types

ORDER BY id;

<strong>6.
Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.</strong>

SElECT u.id,

        COUNT(v_t.name)
        
FROM stackoverflow.users u

JOIN stackoverflow.votes v ON u.id = v.user_id

JOIN stackoverflow.vote_types v_t ON v.vote_type_id = v_t.id

WHERE v_t.name = 'Close'

GROUP BY u.id

order by COUNT(v_t.name) DESC, id DESC

LIMIT 10;

<strong>7.
Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно. Отобразите несколько полей:
идентификатор пользователя;
число значков;
место в рейтинге — чем больше значков, тем выше рейтинг.
Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.</strong>

SELECT u.id, COUNT(b.id), DENSE_RANK() OVER (ORDER BY COUNT(b.id) desc) as rating

FROM stackoverflow.users u

JOIN stackoverflow.badges b ON u.id = b.user_id

WHERE b.creation_date::date between '2008-11-15' AND '2008-12-15'

GROUP BY u.id

ORDER BY COUNT(b.id) DESC, u.id

LIMIT 10;

<strong>8.
Сколько в среднем очков получает пост каждого пользователя?
Сформируйте таблицу из следующих полей:
заголовок поста;
идентификатор пользователя;
число очков поста;
среднее число очков пользователя за пост, округлённое до целого числа.
Не учитывайте посты без заголовка, а также те, что набрали ноль очков.</strong>

SELECT  title,

        user_id,
        
        score,
        
        ROUND(AVG(score) OVER (PARTITION BY user_id))
        
FROM stackoverflow.posts p

WHERE title is not NULL

    AND score != 0;

<strong>9.
Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.</strong>

WITH 

profiles as

(SELECT user_id, COUNT(id)

FROM stackoverflow.badges b

GROUP BY user_id 

HAVING COUNT(id)> 1000)


SELECT  title

FROM  stackoverflow.posts p

WHERE user_id in (

SELECT user_id

FROM profiles)

AND title is not NULL;

<strong>10.
Напишите запрос, который выгрузит данные о пользователях из США (англ. United States). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
пользователям с числом просмотров меньше 100 — группу 3.
Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.</strong>

SELECT id,

       views,
       
       CASE
       
           WHEN views >= 350 THEN 1
           
           WHEN views >= 100 THEN 2
           
           ELSE 3
           
       END as group
       
FROM stackoverflow.users

WHERE location LIKE '%United States%'

 and views != 0

<strong>11.
Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.</strong>

WITH kek AS

(SELECT id,

 CASE
 
           WHEN views >= 350 THEN 1
           
           WHEN views >= 100 AND views < 350 THEN 2
           
           WHEN views < 100 THEN 3
           
       END AS view_,
       
       views
       
FROM stackoverflow.users

WHERE LOCATION LIKE '%United States%' AND views != 0),


lol as (SELECT  *,

        MAX(views) OVER (PARTITION BY view_ ORDER BY view_ DESC)
        
FROM kek)


SELECT id, view_,views

FROM lol

WHERE views = max

ORDER BY views  DESC, id ASC;

<strong>12.
Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
номер дня;
число пользователей, зарегистрированных в этот день;
сумму пользователей с накоплением.</strong>

SELECT  DISTINCT

        EXTRACT(DAY FROM creation_date::date),
        
        COUNT(id) OVER (PARTITION  BY creation_date::date) AS costs,
        
        COUNT(id) OVER (ORDER BY creation_date::date) AS costs_cum
        
FROM stackoverflow.users

WHERE creation_date::date BETWEEN '2008-11-01' and '2008-11-30';

<strong>13.
Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:
идентификатор пользователя;
разницу во времени между регистрацией и первым постом.
</strong>

WITH

tabe as (SELECT  * ,ROW_NUMBER() OVER (PARTITION BY u.id ORDER BY p.creation_date ) as rn,

         u.creation_date as create_user,
         
         p.creation_date as create_post,
         
         u.id as users_idi
         
FROM stackoverflow.posts p

JOIN stackoverflow.users u ON p.user_id = u.id)


SELECT   users_idi,

         create_post - create_user as diff
         
FROM tabe

WHERE rn = 1;

<h2>Вторя часть</h2>

<strong>1.
Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.</strong>

SELECT DATE_TRUNC('month',creation_date)::date,  SUM(views_count)

FROM stackoverflow.posts

GROUP BY DATE_TRUNC('month',creation_date)

ORDER BY  SUM(views_count) DESC;

<strong>2.
Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.</strong>

with t as (

    select
    
    display_name,
    
    count(distinct user_id) as user_ids,  
    
    count(p.id) as post_count

from stackoverflow.users u

join stackoverflow.posts p on u.id = p.user_id

join stackoverflow.post_types pt on p.post_type_id = pt.id

where type = 'Answer' and DATE_TRUNC('day', p.creation_date) <= DATE_TRUNC('day', u.creation_date) + INTERVAL '1 month'

group by display_name
)

select 

    display_name,
    
    user_ids
    
from t
group by display_name, user_ids, post_count

having post_count > 100

order by display_name

<strong>3.
Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.</strong>

WITH 

september AS (SELECT *

FROM stackoverflow.users

WHERE creation_date::date between '2008-09-01' AND '2008-09-30'),


december as (SELECT *

FROM stackoverflow.posts

WHERE creation_date::date between '2008-12-01' AND '2008-12-31'),


use as (SELECT s.id

FROM september s JOIN december d ON s.id = d.user_id)



SELECT  DATE_TRUNC('month',creation_date)::date,

        COUNT(id)
        
FROM stackoverflow.posts

WHERE user_id in (SELECT s.id

FROM september s JOIN december d ON s.id = d.user_id)

GROUP BY DATE_TRUNC('month',creation_date)::date

ORDER BY  DATE_TRUNC('month',creation_date)::date DESC ;


<strong>4.
Используя данные о постах, выведите несколько полей:
идентификатор пользователя, который написал пост;
дата создания поста;
количество просмотров у текущего поста;
сумму просмотров постов автора с накоплением.
Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.</strong>

SELECT   user_id, creation_date, views_count, 

         SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date)
         
FROM stackoverflow.posts;

<strong>5.
Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.</strong>

WITH kek AS (SELECT DISTINCT user_id,

        DATE_TRUNC('day', creation_date)::date as dt
        
FROM stackoverflow.posts

 WHERE DATE_TRUNC('day', creation_date)::date BETWEEN '2008-12-01' AND '2008-12-07'),
 
 lol AS (SELECT  DISTINCT user_id,
 
         COUNT(dt) OVER (PARTITION BY user_id)
         
 FROM kek)
 
 SELECT ROUND(AVG(count))
 
 FROM lol

<strong>6.
На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
номер месяца;
количество постов за месяц;
процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.
Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.
Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.</strong>

 WITH 
 
frst AS

(SELECT DISTINCT EXTRACT(month from creation_date) as dt,

                   COUNT(id) OVER (PARTITION BY DATE_TRUNC('month', creation_date)::date) as count_
                   
FROM stackoverflow.posts

WHERE DATE_TRUNC('day', creation_date)::date BETWEEN '2008-09-01' AND '2008-12-31' 

GROUP BY creation_date,id),

scnd AS 

(SELECT dt,

        CAST(count_ as numeric),
        
        CAST(count_ - LAG(count_, 1) OVER (ORDER BY dt) as numeric) as diff
        
FROM frst)

                       
select dt,

        count_,
        
        ROUND((diff/LAG(count_, 1) OVER (ORDER BY dt))*100, 2)
        
from scnd;


<strong>7.
Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. Выведите данные за октябрь 2008 года в таком виде:
номер недели;
дата и время последнего поста, опубликованного на этой неделе.</strong>

WITH

big_user AS

(SELECT user_id

FROM (SELECT user_id,

       COUNT(id)
       
FROM stackoverflow.posts

GROUP BY user_id

ORDER BY  COUNT(id) DESC

LIMIT 1)as kek)


SELECT DISTINCT  extract(week from creation_date::date) AS week_number,

                 LAST_VALUE(creation_date) OVER (PARTITION BY extract(week from creation_date::date) ORDER BY creation_date 
                 
                 ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING)
                 
FROM stackoverflow.posts

WHERE user_id IN (SELECT*

                  FROM big_user)
                  
                  AND  creation_date between '2008-10-01' and '2008-11-01'